In [88]:
#//*************************************************************************
#//*** Downloads Stock Data and converts the returns to a dataframe,
#//*** which saves a compressed CSV file in the stocks folder
#//*************************************************************************
import json
import requests
import time
import pandas as pd
import platform
import csv

#//*********************************************************************************
#//*** Read the API keys from a JSON encoded file
#//*** Located in the ignore_folder sub directory
#//*** This Folder is added to the .gitignore file and does not show up on Github
#//*** This is Authentication Best Practices for Github
#//*********************************************************************************
f = open("./ignore_folder/alpha_vantage_api.json", "r")

#//*** Fugley Pythonic type conversion
#//*** Loads the file into Dictionary via JSON.loads
#//*** Gets the API key value using the 'api' key
#//*** prepends apikey= so the resulting value is URL ready :]
av_apikey = json.loads(f.read())['apikey']
f.close()

#//*** Load the Stock Tickers
f = open(".\\data\\stock_tickers.json", "r")
symbols = json.loads(f.read())['symbols']

#//*** Just get amc for testing
symbols = [ "amc" ]
f.close()


In [89]:
"""
#//*** Alpha Vantage API Docs:
#//*** https://www.alphavantage.co/documentation/

#//*** Intra day Query
#symbol = "amc"
#url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={av_apikey}'

#//*** get prices throughout today
#action = "TIME_SERIES_INTRADAY"
#action = "TIME_SERIES_DAILY"
#//*** Intraday prices going back two yeares
#action = "TIME_SERIES_INTRADAY_EXTENDED"
#url = f'https://www.alphavantage.co/query?function={action}&symbol={symbol}&interval=60min&slice=year1month1&apikey={av_apikey}'
#url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=60min&slice=year1month3&adjusted=false&apikey={av_apikey}"
"""

#//******************************************************************************
#//*** Builds the URL request based on the symbol and type of data requested.
#//*** Initially, this does the daily numbers.
#//*** Can easily be scaled up to add many different URL request types
#//******************************************************************************
def build_url(input_action,input_symbol,m=1,y=1):
    #//*** Valid Actions:
    #//*******  Daily: Gets the historical daily closing price for up to 20 years
    
    if input_action == 'daily':
        action = "TIME_SERIES_DAILY"
        out = ""
        out += f'https://www.alphavantage.co/query?'
        out += f'function={action}'
        out += f'&symbol={symbol}'
        out += f'&outputsize=full'
        out += f'&apikey={av_apikey}'
        
        return out
    
    if input_action == '1min':
        action = "TIME_SERIES_INTRADAY_EXTENDED"
        out = ""
        out += f'https://www.alphavantage.co/query?'
        out += f'function={action}'
        out += f'&symbol={symbol}'
        out += f'&outputsize=full'
        out += f'&slice=year{y}month{m}'
        out += f'&interval=1min'
        #out += "datatype=json"
        #out += f'&adjusted=true',
        #out += "&slice=year1month1",
        out += f'&apikey={av_apikey}'
        
        return out
    
    
    print(f"Invalid Action: {input_action}")
    print(f"No URL Returned, PLease try again")
    return None
    


action = 'daily'
action = '1min'



for symbol in symbols:
    if action == '1min':
           
        #//*** initialize output dataframe
        out_df = pd.DataFrame()
            
        #//*** Loop the year
        for year in [1,2]:

            #//*** Loop each month
            for month in range(1,13):
                print(f"Length out_df: {len(out_df)}")
                print(f"Building URL: {symbol} - Month {month} Year {year}")
                url = build_url(action,symbol,month,year)

                print("Downloading")
                print(url)
                r = requests.get(url)
                print(r.text[:1000])
                f = open("t.csv", "w")

                f.write(r.text)
                f.close()

                out_df = pd.concat([out_df,pd.read_csv("t.csv")])

                print("Waiting 20 Seconds")
                time.sleep(20)

                    
        print("df Built")
        print(out_df.head(10))

        output_filename = f"./stocks/{symbol}_1min.csv.zip"

        #//*** Convert Path to Mac formatting if needed
        #if platform.system() == 'Darwin':
            #output_filename = output_filename.replace("\\","/")

        print(f"Writing dataframe to File: {output_filename}")
        out_df.to_csv(output_filename,compression="zip",index=False)

                        
                    #print(year,month)
            #print("Waiting 20 Seconds")
            #time.sleep(20)

            #//*** Build the Url Request for each symbol
    #//*** Verify we built a proper url
#    if url != None:
        
#        print("Downloading....")
        #//*** Download the data for each Symbol
#        r = requests.get(url)

"""
        #//*** Convert raw string to dictionary for processing 
        data = r.json()

        #//*** Output Dictionary
        out_dict = {}
        print("Processing....")
        #//*** Process Data into the out_dict
        for date in data[data_key]:
            #//*** Build out_dict (output_dictionary) keys 
            if len(out_dict.keys()) == 0:
                out_dict['date'] = []
                out_dict['symbol'] = []

                #//*** Get this dictionary for the first row. Use the key values, but strip the first 3 characters which are numeric
                for key in data[data_key][date].keys():
                    out_dict[key[3:]] = []

            #//*** Add Date to out_dict
            out_dict['date'].append(date)

            #//*** Add Symbol to out_dict
            out_dict['symbol'].append(symbol)

            #//*** Loop through the daily values and append to the out_dict
            for key,value in data[data_key][date].items():

                #//*** Trim first 3 characters off key and append to the appropriate dictionary list
                out_dict[key[3:]].append(value)

        print("Building Dataframe")
        out_df = pd.DataFrame()
        #//*** Convert the Dictionary to a Dataframe
        #//*** Each Key is a column, the data is the list
        for key,value in out_dict.items():
            out_df[key] = value

        #//*** Generic Filename - Placeholder
        output_filename = f".\\stocks\\{symbol}_need_a_better_name.csv.zip"
        
        
        #//*** Build filename based on action type
        if action == 'daily':
            output_filename = f".\\stocks\\{symbol}_daily.csv.zip"
        
        if action == '1min':
            output_filename = f".\\stocks\\{symbol}_1min.csv.zip"

        #//*** Convert Path to Mac formatting if needed
        if platform.system() == 'Darwin':
            output_filename = output_filename.replace("\\","/")
        
        print(f"Writing dataframe to File: {output_filename}")
        out_df.to_csv(output_filename,compression="zip",index=False)    

    #else:
    #    print("We've got an url problem Skipping")
    
    #//*** Wait 20 seconds so we don't hammer the API
    #//*** Max is 5 calls / minute & 500 /day
    
    #print("Waiting 20 Seconds")
    #time.sleep(20)
"""
print("done")




    

Length out_df: 0
Building URL: amc - Month 1 Year 1
Downloading
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=amc&outputsize=full&slice=year1month1&interval=1min&apikey=S4DI5C1JRKCLIM74
time,open,high,low,close,volume
2021-07-12 20:00:00,41.25,41.34,41.25,41.28,11989
2021-07-12 19:59:00,41.25,41.28,41.25,41.25,8974
2021-07-12 19:58:00,41.33,41.33,41.25,41.25,14056
2021-07-12 19:57:00,41.27,41.35,41.25,41.33,6832
2021-07-12 19:56:00,41.25,41.29,41.22,41.29,10274
2021-07-12 19:55:00,41.25,41.27,41.22,41.22,5550
2021-07-12 19:54:00,41.16,41.23,41.1,41.23,13137
2021-07-12 19:53:00,41.19,41.2,41.15,41.15,8237
2021-07-12 19:52:00,41.15,41.28,41.15,41.19,7237
2021-07-12 19:51:00,41.06,41.2,41.06,41.2,11010
2021-07-12 19:50:00,41.02,41.07,41.01,41.02,17505
2021-07-12 19:49:00,41.03,41.03,41.0,41.02,37562
2021-07-12 19:48:00,41.03,41.03,41.01,41.03,3359
2021-07-12 19:47:00,41.01,41.05,41.0,41.02,8506
2021-07-12 19:46:00,41.1,41.1,41.01,41.01,9780
2021-07-12 19:

Length out_df: 123875
Building URL: amc - Month 8 Year 1
Downloading
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=amc&outputsize=full&slice=year1month8&interval=1min&apikey=S4DI5C1JRKCLIM74
time,open,high,low,close,volume
2020-12-14 20:00:00,3.22,3.22,3.21,3.21,3560
2020-12-14 19:59:00,3.23,3.23,3.22,3.22,1089
2020-12-14 19:57:00,3.21,3.22,3.21,3.21,8840
2020-12-14 19:55:00,3.21,3.2101,3.21,3.2101,3028
2020-12-14 19:52:00,3.22,3.22,3.22,3.22,208
2020-12-14 19:50:00,3.21,3.22,3.21,3.22,500
2020-12-14 19:49:00,3.22,3.22,3.22,3.22,100
2020-12-14 19:47:00,3.22,3.22,3.22,3.22,411
2020-12-14 19:46:00,3.22,3.22,3.22,3.22,480
2020-12-14 19:45:00,3.22,3.22,3.22,3.22,249
2020-12-14 19:44:00,3.23,3.23,3.22,3.22,450
2020-12-14 19:43:00,3.23,3.23,3.22,3.22,1559
2020-12-14 19:41:00,3.23,3.23,3.22,3.22,412
2020-12-14 19:40:00,3.23,3.23,3.23,3.23,500
2020-12-14 19:39:00,3.23,3.23,3.23,3.23,110
2020-12-14 19:38:00,3.23,3.23,3.23,3.23,100
2020-12-14 19:34:00,3.22,3.22,

Length out_df: 204378
Building URL: amc - Month 3 Year 2
Downloading
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=amc&outputsize=full&slice=year2month3&interval=1min&apikey=S4DI5C1JRKCLIM74
time,open,high,low,close,volume
2020-05-18 20:00:00,4.77,4.77,4.77,4.77,10581
2020-05-18 19:58:00,4.77,4.77,4.77,4.77,148
2020-05-18 19:56:00,4.76,4.76,4.76,4.76,652
2020-05-18 19:55:00,4.76,4.76,4.76,4.76,492
2020-05-18 19:54:00,4.75,4.75,4.75,4.75,508
2020-05-18 19:50:00,4.75,4.75,4.75,4.75,302
2020-05-18 19:48:00,4.75,4.75,4.75,4.75,155
2020-05-18 19:46:00,4.75,4.75,4.75,4.75,505
2020-05-18 19:45:00,4.75,4.76,4.75,4.76,1200
2020-05-18 19:44:00,4.75,4.75,4.75,4.75,939
2020-05-18 19:42:00,4.75,4.75,4.75,4.75,150
2020-05-18 19:39:00,4.73,4.73,4.73,4.73,200
2020-05-18 19:35:00,4.75,4.75,4.75,4.75,2007
2020-05-18 19:27:00,4.73,4.73,4.73,4.73,2000
2020-05-18 19:26:00,4.75,4.75,4.75,4.75,300
2020-05-18 19:14:00,4.74,4.74,4.74,4.74,1285
2020-05-18 19:06:00,4.7,4.7,4.7,4

Length out_df: 268078
Building URL: amc - Month 10 Year 2
Downloading
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=amc&outputsize=full&slice=year2month10&interval=1min&apikey=S4DI5C1JRKCLIM74
time,open,high,low,close,volume
2019-10-21 16:23:00,8.93114190899,8.93114190899,8.93114190899,8.93114190899,102
2019-10-21 16:04:00,8.94074205502,8.94074205502,8.94074205502,8.94074205502,1591
2019-10-21 16:03:00,8.94074205502,8.94074205502,8.94074205502,8.94074205502,7422
2019-10-21 16:01:00,8.94074205502,8.94074205502,8.94074205502,8.94074205502,132
2019-10-21 16:00:00,8.94074205502,8.95043917221,8.93104493782,8.94074205502,32294
2019-10-21 15:59:00,8.93589349642,8.95043917221,8.93104493782,8.94054811267,18676
2019-10-21 15:58:00,8.93104493782,8.94074205502,8.93104493782,8.93589349642,13610
2019-10-21 15:57:00,8.95043917221,8.95043917221,8.93104493782,8.93502075587,17858
2019-10-21 15:56:00,8.95043917221,8.95528773081,8.94074205502,8.94559061362,11484
2019-10-2

In [67]:
output_filename = f"./stocks/{symbol}_1min.csv.zip"

#//*** Convert Path to Mac formatting if needed
#if platform.system() == 'Darwin':
    #output_filename = output_filename.replace("\\","/")

print(f"Writing dataframe to File: {output_filename}")
out_df.to_csv(output_filename,compression="zip",index=False)

Writing dataframe to File: ./stocks/gme_1min.csv.zip


In [87]:
tdf = pd.read_csv(output_filename)
print(len(tdf['time'].unique()))
print(len(tdf))

245627
245627


In [49]:
f = open("t.csv", "w")
f.write(r.text)
f.close()

pd.read_csv("t.csv")


#decoded_content = r.decode('utf-8')
#cr = csv.reader(r.text.splitlines(), delimiter=',')
#cr

,time,open,high,low,close,volume
0,2021-07-12 20:00:00,189.50,189.50,189.50,189.50,417
1,2021-07-12 19:53:00,189.44,189.44,189.44,189.44,118
2,2021-07-12 19:22:00,189.01,189.01,189.01,189.01,102
3,2021-07-12 18:52:00,189.00,189.00,189.00,189.00,250
4,2021-07-12 17:53:00,189.50,189.50,189.50,189.50,163
...,...,...,...,...,...,...
10041,2021-06-14 04:38:00,236.39,236.40,236.39,236.40,975
10042,2021-06-14 04:36:00,236.03,236.03,236.03,236.03,358
10043,2021-06-14 04:21:00,237.50,237.50,237.50,237.50,489
10044,2021-06-14 04:15:00,235.95,235.95,235.95,235.95,418


In [71]:
"""
#//**** INTRA day EXTENDED provides stock data at intervals of 1min, 5min, 15min, 30min, 60min,
#//**** Each query provides one month at a time

import csv
CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={symbol}&interval=15min&slice=year1month1&apikey={av_apikey}'

with requests.Session() as s:
    download = s.get(url)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)
"""

"\n#//**** INTRA day EXTENDED provides stock data at intervals of 1min, 5min, 15min, 30min, 60min,\n#//**** Each query provides one month at a time\n\nimport csv\nCSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={symbol}&interval=15min&slice=year1month1&apikey={av_apikey}'\n\nwith requests.Session() as s:\n    download = s.get(url)\n    decoded_content = download.content.decode('utf-8')\n    cr = csv.reader(decoded_content.splitlines(), delimiter=',')\n    my_list = list(cr)\n    for row in my_list:\n        print(row)\n"